In [1]:
import pandas as pd

In [2]:
act_luminica = pd.read_csv('C:/Users/Michael/Documents/ml_zc/datosObtenidos/serie_act_lumincia.csv')
pop1 = pd.read_csv('C:/Users/Michael/Documents/ml_zc/datosObtenidos/poblacion.csv')
pop2 = pd.read_csv('C:/Users/Michael/Documents/ml_zc/datosObtenidos/poblacion_21.csv')
urb = pd.read_csv('C:/Users/Michael/Documents/ml_zc/datosObtenidos/urbanidad.csv')
nbi = pd.read_excel('C:/Users/Michael/Documents/ml_zc/datosObtenidos/NBI_parroquia.xlsx', sheet_name='data')
carreteras = pd.read_csv('C:/Users/Michael/Documents/ml_zc/datosObtenidos/carreteras.csv')
redes = pd.read_csv('C:/Users/Michael/Documents/ml_zc/datosObtenidos/analisis_redes.csv')
redes1 = pd.read_csv('C:/Users/Michael/Documents/ml_zc/datosObtenidos/analisis_redes_2.csv')
servicios = pd.read_csv('C:/Users/Michael/Documents/ml_zc/datosObtenidos/servicios.csv')
n_educativos = pd.read_csv('C:/Users/Michael/Documents/ml_zc/datosObtenidos/num_educativos.csv')
ndvi1 = pd.read_csv('C:/Users/Michael/Documents/ml_zc/datosObtenidos/nvdi_22.csv')
ndvi2 = pd.read_csv('C:/Users/Michael/Documents/ml_zc/datosObtenidos/nvdif2.csv')
mndwi1 = pd.read_csv('C:/Users/Michael/Documents/ml_zc/datosObtenidos/mndwi.csv')
mndwi2 = pd.read_csv('C:/Users/Michael/Documents/ml_zc/datosObtenidos/mndwi2.csv')

In [3]:
pop = pd.concat([pop1, pop2], ignore_index=True).drop_duplicates()

In [4]:
redes = pd.concat([redes, redes1], ignore_index=True).drop_duplicates()

In [5]:
# Comprobar la serie
# pop['ADM3_PCODE'].value_counts().reset_index().rename(columns={'count': 'conteo'})['conteo'].unique()

In [6]:
#t = pop['ADM3_PCODE'].value_counts().reset_index().rename(columns={'count': 'conteo'})
#t

In [7]:
edi1 = pd.read_csv('C:/Users/Michael/Documents/ml_zc/datosObtenidos/edificaciones.csv')
edi2 = pd.read_csv('C:/Users/Michael/Documents/ml_zc/datosObtenidos/edificaciones_2.csv')
edi3 = pd.read_csv('C:/Users/Michael/Documents/ml_zc/datosObtenidos/edificaciones_3.csv')

edi = pd.concat([edi1, edi2], ignore_index=True).drop_duplicates()
edi = pd.concat([edi, edi3], ignore_index=True).drop_duplicates()

In [8]:
ndvi = pd.concat([ndvi1, ndvi2], ignore_index=True).drop_duplicates()

mndwi = pd.concat([mndwi1, mndwi2], ignore_index=True).drop_duplicates()

In [9]:
edi['timestamp'] = edi['timestamp'].astype(str).str[:4].astype(int)
edi = edi.rename(columns={'timestamp': 'Año', 'value': 'edificaciones'})

carreteras['timestamp'] = carreteras['timestamp'].astype(str).str[:4].astype(int)
carreteras = carreteras.rename(columns={'timestamp': 'Año', 'value': 'carreteras'})

#n_agua['timestamp'] = n_agua['timestamp'].astype(str).str[:4].astype(int)
#n_agua = n_agua.rename(columns={'timestamp': 'Año'})

n_educativos['timestamp'] = n_educativos['timestamp'].astype(str).str[:4].astype(int)
n_educativos = n_educativos.rename(columns={'timestamp': 'Año'})

In [10]:
df = pd.merge(act_luminica, pop, how='left', on=['ADM3_PCODE', 'Año'])
df = pd.merge(df, urb, how='left', on=['ADM3_PCODE', 'Año'])
df = pd.merge(df, edi, how='left', on=['ADM3_PCODE', 'Año'])
df = pd.merge(df, carreteras, how='left', on=['ADM3_PCODE', 'Año'])
df = pd.merge(df, redes, how='left', on=['ADM3_PCODE', 'Año'])
df = pd.merge(df, servicios, how='left', on=['ADM3_PCODE', 'Año'])
#df = pd.merge(df, n_agua, how='left', on=['ADM3_PCODE', 'Año'])
df = pd.merge(df, n_educativos, how='left', on=['ADM3_PCODE', 'Año'])
df = pd.merge(df, ndvi, how='left', on=['ADM3_PCODE', 'Año'])
df = pd.merge(df, mndwi, how='left', on=['ADM3_PCODE', 'Año'])

In [11]:
# Añadir área total del polígono
# Cargar shapefile local
import geopandas as gpd

shapefile_path = "C:/Users/Michael/Documents/ml_zc/ecu_adm_2024/ecu_adm_adm3_2024.shp"
gdf = gpd.read_file(shapefile_path)

In [12]:
gdf = gdf.to_crs(epsg=32717)
gdf['area'] = gdf['geometry'].area

In [13]:
df = pd.merge(df, gdf[['ADM3_PCODE', 'area']], how='left', on=['ADM3_PCODE'])

# Índice
Construcción: <br>
1. Obtener la distribución de luz por persona.
2. Obtener la distribución de construcción por metro cuadrado de uso urbano.
3. Otener la distribución de luz individual por metro ocupado en construcción.
Es importante transformar los $cm^2$ del VIIR a $m^2$, así que multiplicamos por 10 000 con este fin.

In [14]:
#df['icl'] = (1e4*df['avg_rad_max']/df['pop'])/(df['edificaciones']/df['m2'])

In [15]:
df.to_csv('C:/Users/Michael/Documents/ml_zc/datosObtenidos/geoespaciales.csv', index_label=False)